In [ ]:
#source "data science from scratch"

In [25]:
import math, random, re
from collections import defaultdict, Counter
from bs4 import BeautifulSoup
import requests
from nltk.corpus import PlaintextCorpusReader
import os.path
import sys

basepath = os.path.dirname('/Users/TheVarunVaria/Downloads/')
corpus_root= os.path.abspath(os.path.join(basepath, "Text"))
wordlists=PlaintextCorpusReader(corpus_root, '.*')

In [26]:
# Gibbs Sampling
#

def roll_a_die():
    return random.choice([1,2,3,4,5,6])

def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

def random_y_given_x(x):
    """equally likely to be x + 1, x + 2, ... , x + 6"""
    return x + roll_a_die()

def random_x_given_y(y):
    if y <= 7:
        # if the total is 7 or less, the first die is equally likely to be
        # 1, 2, ..., (total - 1)
        return random.randrange(1, y)
    else:
        # if the total is 7 or more, the first die is equally likely to be
        # (total - 6), (total - 5), ..., 6
        return random.randrange(y - 6, 7)

def gibbs_sample(num_iters=100):
    x, y = 1, 2 # doesn't really matter
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

def compare_distributions(num_samples=1000):
    counts = defaultdict(lambda: [0, 0])
    for _ in range(num_samples):
        counts[gibbs_sample()][0] += 1
        counts[direct_sample()][1] += 1
    return counts

In [27]:
#
# TOPIC MODELING
#

def sample_from(weights):
    total = sum(weights)
    rnd = total * random.random()       # uniform between 0 and total
    for i, w in enumerate(weights):
        rnd -= w                        # return the smallest i such that
        if rnd <= 0: return i           # sum(weights[:(i+1)]) >= rnd
document= []
for fileid in wordlists.fileids():
    document.append(fileid)
print(len(document))

162


In [28]:
K = 4

document_topic_counts = [Counter()
                         for _ in documents]

topic_word_counts = [Counter() for _ in range(K)]

topic_counts = [0 for _ in range(K)]

document_lengths = [len(d) for d in documents]

distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)

D = len(documents)

def p_topic_given_document(topic, d, alpha=0.1):
    """the fraction of words in document _d_
    that are assigned to _topic_ (plus some smoothing)"""

    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    """the fraction of words assigned to _topic_
    that equal _word_ (plus some smoothing)"""

    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

def topic_weight(d, word, k):
    """given a document and a word in that document,
    return the weight for the k-th topic"""

    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k)
                        for k in range(K)])

In [29]:
random.seed(0)
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):

            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [30]:
if __name__ == "__main__":
    print("gibbs sampling")
    comparison = compare_distributions()
    for roll, (gibbs, direct) in comparison.items():
        print(roll, gibbs, direct)


    # topic MODELING

    for k, word_counts in enumerate(topic_word_counts):
        for word, count in word_counts.most_common():
            if count > 0: print(k, word, count)

    topic_names = ["Customer Service",
                   "Business",
                   "Stock",
                   "Management"]

    for document, topic_counts in zip(documents, document_topic_counts):
        print(document)
        for topic, count in topic_counts.most_common():
            if count > 0:
                print(topic_names[topic], count)
        print()

gibbs sampling
(3, 5) 38 23
(3, 8) 24 32
(4, 8) 21 32
(1, 4) 21 37
(5, 6) 29 34
(3, 9) 27 24
(1, 2) 25 24
(1, 6) 37 29
(2, 4) 22 19
(4, 6) 30 23
(6, 11) 24 29
(6, 12) 29 34
(2, 6) 35 22
(2, 8) 22 26
(2, 5) 23 35
(6, 7) 25 32
(5, 10) 25 26
(5, 7) 34 29
(3, 4) 17 29
(4, 5) 36 23
(3, 7) 28 36
(4, 9) 30 40
(5, 9) 31 27
(3, 6) 26 21
(2, 7) 36 29
(6, 10) 21 22
(6, 9) 28 25
(4, 7) 30 22
(5, 8) 31 32
(4, 10) 25 26
(6, 8) 34 27
(1, 5) 32 29
(1, 3) 31 20
(1, 7) 26 32
(5, 11) 26 26
(2, 3) 21 24
